In [20]:
import pandas as pd

# Read the CSV file
gpa_data = pd.read_csv('gpa.csv')

# Exclude courses from the year 2024
gpa_data = gpa_data[gpa_data['Year'] != 2024]

# Group by 'Course Title' and 'Year', then check if all terms are present
course_year_group = gpa_data.groupby(['Course Title', 'Year'])['Term'].nunique().reset_index()

# Filter the courses that have all four terms
courses_all_terms = course_year_group[course_year_group['Term'] == 4]

# Merge the filtered courses back with the original data to keep all columns
filtered_gpa_data = gpa_data.merge(courses_all_terms[['Course Title', 'Year']], on=['Course Title', 'Year'], how='inner')

filtered_gpa_data

,Year,Term,YearTerm,Subject,Number,Course Title,Sched Type,A+,A,A-,...,C+,C,C-,D+,D,D-,F,W,Primary Instructor,Students
0,2023,Fall,2023-fa,DANC,100,Intro to Contemporary Dance,LAB,0,19,3,...,0,0,1,0,0,0,3,0,"Allen, Betty J",29
1,2023,Fall,2023-fa,DANC,100,Intro to Contemporary Dance,ONL,0,16,2,...,0,0,0,0,0,0,0,0,"Owens, Nikolas C",23
2,2023,Fall,2023-fa,DANC,100,Intro to Contemporary Dance,ONL,1,15,5,...,0,0,0,0,1,0,1,0,"Condon, Elizabeth G",24
3,2023,Fall,2023-fa,DANC,100,Intro to Contemporary Dance,ONL,0,18,3,...,0,0,2,1,0,0,0,0,"Segale, Magdalyn L",25
4,2023,Fall,2023-fa,DANC,100,Intro to Contemporary Dance,ONL,17,5,1,...,0,0,0,0,0,0,0,0,"Gonzalez, Gabriel B",25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2911,2015,Winter,2015-wi,BADM,350,IT for Networked Organizations,ONL,2,8,2,...,1,2,2,1,0,0,2,0,"Sachdev, Vishal",28
2912,2015,Winter,2015-wi,BADM,380,International Business,ONL,0,9,12,...,0,2,0,3,0,0,0,0,"Bucheli, Marcelo",42
2913,2015,Winter,2015-wi,ECON,102,Microeconomic Principles,ONL,10,71,9,...,3,0,0,0,0,0,2,0,"Vazquez, Jose J",104
2914,2015,Winter,2015-wi,ECON,203,Economic Statistics II,ONL,10,6,9,...,7,5,0,2,2,0,5,1,"Petry, Joseph A",57


In [21]:
# Define a dictionary to map letter grades to numeric values
grade_mapping = {'A+': 1.0, 'A': 1.0, 'A-': 1.3, 'B+': 1.7, 'B': 2.0, 'B-': 2.3, 'C+': 2.7, 'C': 3.0, 'C-': 3.3, 'D+': 3.7, 'D': 4.0, 'D-': 4.3, 'F': 5.0}

# Calculate the total numeric grade for each row
filtered_gpa_data['Average Numeric Grade'] = (
    (filtered_gpa_data['A+'] * grade_mapping['A+'] +
    filtered_gpa_data['A'] * grade_mapping['A'] +
    filtered_gpa_data['A-'] * grade_mapping['A-'] +
    filtered_gpa_data['B+'] * grade_mapping['B+'] +
    filtered_gpa_data['B'] * grade_mapping['B'] +
    filtered_gpa_data['B-'] * grade_mapping['B-'] +
    filtered_gpa_data['C+'] * grade_mapping['C+'] +
    filtered_gpa_data['C'] * grade_mapping['C'] +
    filtered_gpa_data['C-'] * grade_mapping['C-'] +
    filtered_gpa_data['D+'] * grade_mapping['D+'] +
    filtered_gpa_data['D'] * grade_mapping['D'] +
    filtered_gpa_data['D-'] * grade_mapping['D-'] +
    filtered_gpa_data['F'] * grade_mapping['F'] )
    / filtered_gpa_data[['A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D', 'D-', 'F']].sum(axis=1)
)


# Calculate the average numeric grade for each row

filtered_gpa_data


,Year,Term,YearTerm,Subject,Number,Course Title,Sched Type,A+,A,A-,...,C,C-,D+,D,D-,F,W,Primary Instructor,Students,Average Numeric Grade
0,2023,Fall,2023-fa,DANC,100,Intro to Contemporary Dance,LAB,0,19,3,...,0,1,0,0,0,3,0,"Allen, Betty J",29,1.596552
1,2023,Fall,2023-fa,DANC,100,Intro to Contemporary Dance,ONL,0,16,2,...,0,0,0,0,0,0,0,"Owens, Nikolas C",23,1.217391
2,2023,Fall,2023-fa,DANC,100,Intro to Contemporary Dance,ONL,1,15,5,...,0,0,0,1,0,1,0,"Condon, Elizabeth G",24,1.395833
3,2023,Fall,2023-fa,DANC,100,Intro to Contemporary Dance,ONL,0,18,3,...,0,2,1,0,0,0,0,"Segale, Magdalyn L",25,1.368000
4,2023,Fall,2023-fa,DANC,100,Intro to Contemporary Dance,ONL,17,5,1,...,0,0,0,0,0,0,0,"Gonzalez, Gabriel B",25,1.080000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2911,2015,Winter,2015-wi,BADM,350,IT for Networked Organizations,ONL,2,8,2,...,2,2,1,0,0,2,0,"Sachdev, Vishal",28,2.003571
2912,2015,Winter,2015-wi,BADM,380,International Business,ONL,0,9,12,...,2,0,3,0,0,0,0,"Bucheli, Marcelo",42,1.754762
2913,2015,Winter,2015-wi,ECON,102,Microeconomic Principles,ONL,10,71,9,...,0,0,0,0,0,2,0,"Vazquez, Jose J",104,1.226923
2914,2015,Winter,2015-wi,ECON,203,Economic Statistics II,ONL,10,6,9,...,5,0,2,2,0,5,1,"Petry, Joseph A",57,2.149123
